In [ ]:
# ! pip install -U FlagEmbedding  # Need this

In [1]:
from FlagEmbedding import FlagReranker
import re

/home/sharchit/anaconda3/envs/devrev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def extract_api_info(unique_docs):
    api_info_list = []
    for idx,i in enumerate(unique_docs):
        content = i.page_content
        api_name_match = re.search(r'##API Name: (.+)', i.page_content)
        description_match = re.search(r'###Description: (.+?)(?=\n\n|\Z)', i.page_content, re.DOTALL)
        arguments_match = re.findall(r'API Argumet: (.+?)\nArgument Description: (.+?)\nReturn Type: (.+?)(?=\n\n|\Z)', i.page_content, re.DOTALL)

        formatted_string = "{}\n{}\n{}".format(api_name_match.group(0), description_match.group(0), '\n'.join([f'{arg[0]}\nArgument Description: {arg[1]}\nReturn Type: {arg[2]}' for arg in arguments_match]))        
        
        struct = {"id":idx, "passage":formatted_string}
        # print(struct)
        api_info_list.append(struct)
        
    return api_info_list

In [3]:
def filter_api_names(unique_docs):
    api_names = []
    for i in unique_docs:
        print((i.page_content))
        api_match = re.search(r'##API Name: (.+)', i.page_content)
        if api_match:
            api_names.append(api_match.group(1).strip())
    return api_names

In [4]:
reranker = FlagReranker('BAAI/bge-reranker-base', use_fp16=False) 

tokenizer_config.json: 100%|██████████| 443/443 [00:00<00:00, 131kB/s]
sentencepiece.bpe.model: 100%|██████████| 5.07M/5.07M [00:00<00:00, 18.0MB/s]
config.json: 100%|██████████| 799/799 [00:00<00:00, 1.54MB/s]
model.safetensors: 100%|██████████| 1.11G/1.11G [00:44<00:00, 25.0MB/s]


In [ ]:
def re_rank(passages_with_id_list, query, top_k):

    passages_with_id_and_scores_list = []
    for passage_with_id in passages_with_id_list:
        score = reranker.compute_score([query, passage_with_id["passage"]])
        new_struct = {"id": passage_with_id["id"], "passage": passage_with_id["passage"], "score": score}
        passages_with_id_and_scores_list.append(new_struct)
    
    passages_with_id_and_scores_list = sorted(passages_with_id_and_scores_list, key=lambda x: x["score"], reverse=True)
    if len(passages_with_id_and_scores_list) <= top_k:
        return passages_with_id_and_scores_list
    else:
        return passages_with_id_and_scores_list[:top_k]

In [ ]:
top_k = len(unique_docs_hf) + 2
query = query_handler.generated_queries

passages = extract_api_info(unique_docs_hf)
re_ranked_passages = re_rank(passages, query, top_k)

retrieved_list = []
for p in re_ranked_passages:
    api_des = p["passage"]
    retrieved_list.append(api_des)